# Aspect Extraction

In [53]:
from pprint import pprint # pretty print


In [54]:
# We get started by importing spacy
import spacy
nlp = spacy.load("en_core_web_sm")

In [55]:
sentences = [
  'The food we had yesterday was delicious',
  'My time in Italy was very enjoyable',
  'I found the meal to be tasty',
  'The internet was slow.',
  'Our experience was suboptimal',
  'anc work in the first 3 days but the proximity sensor on the left bud stopped working',
  'shipped out immediately, received the item in good condition, no damage well packed, thank you shopee and thank you seller, will order again 👍😁👌🫰',
  "Tysm for delivering my items safely. I only expected the seller gave some freebies, but he/she didn't 😂😂😾",
  'Sobrang tagal lang dumating. Pero okay lang, sulit naman. Nakuha ko sya sa halagang 999 pesos. Nagsale kasi. 😍 Thank youuu! Legit !',
  "been using this buds for 2 weeks na and I notice that sometimes it took so long to connect on my laptop and phone I don't know why. then sometimes nagkaproblem sound nya parang basag yung tunog",
  'wrong color. poor packaging',
  'Delivery and shipment were both okay.',
  """If you intend to use this to attract your man, be prepared for a ravenous, passionate result. Do not use if you don’t intend to follow through with your intended target. That’s a d*** move. Plus this stuff is potent and you have no idea how a man might respond to its siren call. It’s basically a love potion so handle with care. But it absolutely works!
Tip 1: let it settle after application, it’s strong but nice smelling at first, but mellows out over time.
Tip 2: combine with your perfume of choice, particularly one that has a positive olfactory memory trigger with your partner/intended target.
Tip 3: don’t use too much, or it becomes a turn OFF (tested and confirmed)! A dab or small swipe on wrists and chest or behind ear/jaw should do it
Worth the money. Have fun!"""
]

In [56]:
from spacy import displacy
testtext = nlp(sentences[-1])
displacy.render(testtext, style = "dep")

In [57]:
displacy.render(testtext, style = "ent")

In [58]:
displacy.render(testtext, style='dep', options={'compact': True})

For each token inside our sentences, we can see the dependency thanks to spacy’s dependency parsing and the POS (Part-Of-Speech) tags. We’re also paying attention to the child tokens, so that we’re able to pick up intensifiers such as “very”, “quite”, and more.

In [59]:
for sentence in sentences:
  doc = nlp(sentence)
  for token in doc:
    print(token.text, spacy.explain(token.dep_), token.head.text, spacy.explain(token.head.pos_),
      token.pos_,[child for child in token.children], sep=' | ')

The | determiner | food | noun | DET | []
food | nominal subject | was | auxiliary | NOUN | [The, had]
we | nominal subject | had | verb | PRON | []
had | relative clause modifier | food | noun | VERB | [we, yesterday]
yesterday | noun phrase as adverbial modifier | had | verb | NOUN | []
was | root | was | auxiliary | AUX | [food, delicious]
delicious | adjectival complement | was | auxiliary | ADJ | []
My | possession modifier | time | noun | PRON | []
time | nominal subject | was | auxiliary | NOUN | [My, in]
in | prepositional modifier | time | noun | ADP | [Italy]
Italy | object of preposition | in | adposition | PROPN | []
was | root | was | auxiliary | AUX | [time, enjoyable]
very | adverbial modifier | enjoyable | adjective | ADV | []
enjoyable | adjectival complement | was | auxiliary | ADJ | [very]
I | nominal subject | found | verb | PRON | []
found | root | found | verb | VERB | [I, be]
the | determiner | meal | noun | DET | []
meal | nominal subject | be | auxiliary | NOUN

Let’s see how to pick up the sentiment descriptions first.

In [60]:
for sentence in sentences:
  doc = nlp(sentence)
  descriptive_term = ''
  for token in doc:
    if token.pos_ == 'ADJ':
      descriptive_term = token
  print(sentence)
  print(f'>{descriptive_term}')

The food we had yesterday was delicious
>delicious
My time in Italy was very enjoyable
>enjoyable
I found the meal to be tasty
>tasty
The internet was slow.
>slow
Our experience was suboptimal
>suboptimal
anc work in the first 3 days but the proximity sensor on the left bud stopped working
>left
shipped out immediately, received the item in good condition, no damage well packed, thank you shopee and thank you seller, will order again 👍😁👌🫰
>good
Tysm for delivering my items safely. I only expected the seller gave some freebies, but he/she didn't 😂😂😾
>
Sobrang tagal lang dumating. Pero okay lang, sulit naman. Nakuha ko sya sa halagang 999 pesos. Nagsale kasi. 😍 Thank youuu! Legit !
>sulit
been using this buds for 2 weeks na and I notice that sometimes it took so long to connect on my laptop and phone I don't know why. then sometimes nagkaproblem sound nya parang basag yung tunog
>
wrong color. poor packaging
>poor
Delivery and shipment were both okay.
>okay
If you intend to use this to a

In [61]:
for sentence in sentences:
  doc = nlp(sentence)
  descriptive_term = ''
  for token in doc:
    if token.pos_ == 'ADJ':
      prepend = ''
      for child in token.children:
        if child.pos_ != 'ADV':
          continue
        prepend += child.text + ' '
      descriptive_term = prepend + token.text
  print(sentence)
  print(f'>{descriptive_term}')

The food we had yesterday was delicious
>delicious
My time in Italy was very enjoyable
>very enjoyable
I found the meal to be tasty
>tasty
The internet was slow.
>slow
Our experience was suboptimal
>suboptimal
anc work in the first 3 days but the proximity sensor on the left bud stopped working
>left
shipped out immediately, received the item in good condition, no damage well packed, thank you shopee and thank you seller, will order again 👍😁👌🫰
>good
Tysm for delivering my items safely. I only expected the seller gave some freebies, but he/she didn't 😂😂😾
>
Sobrang tagal lang dumating. Pero okay lang, sulit naman. Nakuha ko sya sa halagang 999 pesos. Nagsale kasi. 😍 Thank youuu! Legit !
>sulit
been using this buds for 2 weeks na and I notice that sometimes it took so long to connect on my laptop and phone I don't know why. then sometimes nagkaproblem sound nya parang basag yung tunog
>
wrong color. poor packaging
>poor
Delivery and shipment were both okay.
>okay
If you intend to use this

As you can see, this time around we picked up very enjoyable as well. Our simplistic algorithm is able to pick up adverbs. It checks for child tokens for each adjective and picks up the adverbs such as “very”, “quite”, etc.

In [62]:
aspects = []
for sentence in sentences:
  doc = nlp(sentence)
  descriptive_term = ''
  target = ''
  for token in doc:
    if token.dep_ == 'nsubj' and token.pos_ == 'NOUN':
      target = token.text
    if token.pos_ == 'ADJ':
      prepend = ''
      for child in token.children:
        if child.pos_ != 'ADV':
          continue
        prepend += child.text + ' '
      descriptive_term = prepend + token.text
  aspects.append({'aspect': target,
    'description': descriptive_term})

pprint(aspects)

[{'aspect': 'food', 'description': 'delicious'},
 {'aspect': 'time', 'description': 'very enjoyable'},
 {'aspect': 'meal', 'description': 'tasty'},
 {'aspect': 'internet', 'description': 'slow'},
 {'aspect': 'experience', 'description': 'suboptimal'},
 {'aspect': 'sensor', 'description': 'left'},
 {'aspect': 'damage', 'description': 'good'},
 {'aspect': 'seller', 'description': ''},
 {'aspect': '', 'description': 'sulit'},
 {'aspect': '', 'description': ''},
 {'aspect': '', 'description': 'poor'},
 {'aspect': 'Delivery', 'description': 'okay'},
 {'aspect': 'dab', 'description': 'Worth'}]


Now our solution is starting to look more complete. We’re able to pick up aspects, even though our application doesn’t “know” anything beforehand. We haven’t hardcoded the aspects such as “food”, “time”, or “meal”. And we also haven’t hardcoded the adjectives such as “tasty”, “slow”, or “enjoyable”.

# Sentiment Analysis
Now that we successfully extracted the aspects and descriptions, it’s time to classify them as positive or negative. The goal here is to help the computer understand that tasty food is positive, while slow internet is negative. Computers don’t understand English, so we will need to try a few things before we have a working solution.

In [63]:
# %pip install textblob

In [64]:
from textblob import TextBlob
for aspect in aspects:
  aspect['sentiment'] = TextBlob(aspect['description']).sentiment
  
pprint(aspects)

[{'aspect': 'food',
  'description': 'delicious',
  'sentiment': Sentiment(polarity=1.0, subjectivity=1.0)},
 {'aspect': 'time',
  'description': 'very enjoyable',
  'sentiment': Sentiment(polarity=0.65, subjectivity=0.78)},
 {'aspect': 'meal',
  'description': 'tasty',
  'sentiment': Sentiment(polarity=0.0, subjectivity=0.0)},
 {'aspect': 'internet',
  'description': 'slow',
  'sentiment': Sentiment(polarity=-0.30000000000000004, subjectivity=0.39999999999999997)},
 {'aspect': 'experience',
  'description': 'suboptimal',
  'sentiment': Sentiment(polarity=0.0, subjectivity=0.0)},
 {'aspect': 'sensor',
  'description': 'left',
  'sentiment': Sentiment(polarity=0.0, subjectivity=0.0)},
 {'aspect': 'damage',
  'description': 'good',
  'sentiment': Sentiment(polarity=0.7, subjectivity=0.6000000000000001)},
 {'aspect': 'seller',
  'description': '',
  'sentiment': Sentiment(polarity=0.0, subjectivity=0.0)},
 {'aspect': '',
  'description': 'sulit',
  'sentiment': Sentiment(polarity=0.0, sub

In [65]:
for i, sentence in enumerate(sentences):
    print(f'{i+1:02}). {sentence}')

01). The food we had yesterday was delicious
02). My time in Italy was very enjoyable
03). I found the meal to be tasty
04). The internet was slow.
05). Our experience was suboptimal
06). anc work in the first 3 days but the proximity sensor on the left bud stopped working
07). shipped out immediately, received the item in good condition, no damage well packed, thank you shopee and thank you seller, will order again 👍😁👌🫰
08). Tysm for delivering my items safely. I only expected the seller gave some freebies, but he/she didn't 😂😂😾
09). Sobrang tagal lang dumating. Pero okay lang, sulit naman. Nakuha ko sya sa halagang 999 pesos. Nagsale kasi. 😍 Thank youuu! Legit !
10). been using this buds for 2 weeks na and I notice that sometimes it took so long to connect on my laptop and phone I don't know why. then sometimes nagkaproblem sound nya parang basag yung tunog
11). wrong color. poor packaging
12). Delivery and shipment were both okay.
13). If you intend to use this to attract your man, 

In [66]:
for i, aspect in enumerate(aspects):
    print(f'{i+1:02}.) "{sentences[i]}"')
    print(f'\tASPECT:"{aspect["aspect"]}",',
            f'DESCRIPTION:"{aspect["description"]}",',
            f'POLARITY:{"POSITIVE" if (aspect["sentiment"].polarity) > 0 else "NEGATIVE" if (aspect["sentiment"].polarity) < 0 else "NEUTRAL"}')
    print(f'\t"{aspect["sentiment"]}"\n')

01.) "The food we had yesterday was delicious"
	ASPECT:"food", DESCRIPTION:"delicious", POLARITY:POSITIVE
	"Sentiment(polarity=1.0, subjectivity=1.0)"

02.) "My time in Italy was very enjoyable"
	ASPECT:"time", DESCRIPTION:"very enjoyable", POLARITY:POSITIVE
	"Sentiment(polarity=0.65, subjectivity=0.78)"

03.) "I found the meal to be tasty"
	ASPECT:"meal", DESCRIPTION:"tasty", POLARITY:NEUTRAL
	"Sentiment(polarity=0.0, subjectivity=0.0)"

04.) "The internet was slow."
	ASPECT:"internet", DESCRIPTION:"slow", POLARITY:NEGATIVE
	"Sentiment(polarity=-0.30000000000000004, subjectivity=0.39999999999999997)"

05.) "Our experience was suboptimal"
	ASPECT:"experience", DESCRIPTION:"suboptimal", POLARITY:NEUTRAL
	"Sentiment(polarity=0.0, subjectivity=0.0)"

06.) "anc work in the first 3 days but the proximity sensor on the left bud stopped working"
	ASPECT:"sensor", DESCRIPTION:"left", POLARITY:NEUTRAL
	"Sentiment(polarity=0.0, subjectivity=0.0)"

07.) "shipped out immediately, received the item

# ASPECT EXTRACTION with CUSTOM SENTIMENT ANALYSIS NAIVE BAYES

In [67]:
# Importing libraries
import pandas as pd
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import pickle

model = pickle.load(open('modelNB.pkl', 'rb'))

# @app.route('/')
# def home():
#     return render_template('index.html')

def predict(raw_text: str):

    # Instantiate PorterStemmer
    p_stemmer = PorterStemmer()

    # Remove HTML
    review_text = BeautifulSoup(raw_text, features="lxml").get_text()

    # Remove non-letters
    letters_only = re.sub("[^a-zA-Z]", " ", review_text)

    # Convert words to lower case and split each word up
    words = letters_only.lower().split()

    # Convert stopwords to a set
    stops = set(stopwords.words('english'))

    # Adding on stopwords that were appearing frequently in both positive and negative reviews
    stops.update(['app','shopee','shoppee','item','items','seller','sellers','bad'])

    # Remove stopwords
    meaningful_words = [w for w in words if w not in stops]

    # Stem words
    meaningful_words = [p_stemmer.stem(w) for w in meaningful_words]

    # Join words back into one string, with a space in between each word
    final_text = pd.Series(" ".join(meaningful_words))

    # Generate predictions
    pred = model.predict(final_text)[0]

    if pred == 1:
        output = "Negative"
    else:
        output = "Postive"

    return output


text = """
anc work in the first 3 days but the proximity sensor on the left bud stopped working
"""
result = predict(text)
print(f'RAW_TEXT: \t{text}')
print(F'\tSENTIMENT: {result}')

RAW_TEXT: 	
anc work in the first 3 days but the proximity sensor on the left bud stopped working

	SENTIMENT: Negative


In [68]:
for i, aspect in enumerate(aspects):
    print(f'{i+1:02}.) "{sentences[i]}"')
    print(f'\tASPECT:"{aspect["aspect"]}",',
            f'DESCRIPTION:"{aspect["description"]}",',
            f'SENTIMENT: {predict(aspect["description"])}'
    )

01.) "The food we had yesterday was delicious"
	ASPECT:"food", DESCRIPTION:"delicious", SENTIMENT: Postive
02.) "My time in Italy was very enjoyable"
	ASPECT:"time", DESCRIPTION:"very enjoyable", SENTIMENT: Postive
03.) "I found the meal to be tasty"
	ASPECT:"meal", DESCRIPTION:"tasty", SENTIMENT: Postive
04.) "The internet was slow."
	ASPECT:"internet", DESCRIPTION:"slow", SENTIMENT: Negative
05.) "Our experience was suboptimal"
	ASPECT:"experience", DESCRIPTION:"suboptimal", SENTIMENT: Postive
06.) "anc work in the first 3 days but the proximity sensor on the left bud stopped working"
	ASPECT:"sensor", DESCRIPTION:"left", SENTIMENT: Postive
07.) "shipped out immediately, received the item in good condition, no damage well packed, thank you shopee and thank you seller, will order again 👍😁👌🫰"
	ASPECT:"damage", DESCRIPTION:"good", SENTIMENT: Postive
08.) "Tysm for delivering my items safely. I only expected the seller gave some freebies, but he/she didn't 😂😂😾"
	ASPECT:"seller", DESCRIPT